In [1]:
from lipydomics import interactive

In [3]:
interactive.main()


What would you like to do?
	1. Make a new Dataset
	2. Load a previous Dataset
> 2
Please enter the path to the pickle file you want to load.
> sample_dset.pickle
! INFO: Loaded existing Dataset from .pickle file: "sample_dset.pickle"

What would you like to do with this Dataset? 
	1. Manage Groups
	2. Filter Data
	3. Manage Statistics
	4. Make Plots
	5. Lipid Identification
	6. Normalize Intensities
	7. Overview of Dataset
	8. Export Current Dataset to Spreadsheet
	9. Save Current Dataset to File
	"exit" to quit the interface
> 7
Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=20,
	features=773,
	identified=414,
	normalized=True,
	group_indices={
		"A": [0, 1, 2, 3]
		"B": [4, 5, 6, 7]
		"C": [8, 9, 10, 11]
		"D": [12, 13, 14, 15]
		"E": [16, 17, 18, 19]
	},
	stats={}
)

What would you like to do with this Dataset? 
	1. Manage Groups
	2. Filter Data
	3. Manage Statistics
	4. Make Plots
	5. Lipid Identification
	6. Normalize Intensities
	7. Overview of Dataset
	8. Export Cur